# Using AWS Rekognition
AWS Rekognition is Amazon's deep learning image and video analysis service. In this file, you'll learn how to call Rekognition from within SageMaker Studio using files stored in S3. Amazon Rekognition has pre-trained models to identify objects, people, text, etc.  

In this file we will do the following:
* Import image data from the web and save to S3
* Use Rekognition to detect people and the dominant emotions they display
* Convert results from Rekognition into a Pandas dataframe that we can use in our analysis
* Use Rekognition to detect and label a variety of objects in the images we process
* Use Rekognition to identify text found in images

### Importing Image Data and Saving it to S3

In this section we will download a collection of images from the internet and then upload them to S3. In later sections we will refer to these files on S3, so let's create a couple of variables to define the bucket and the prefix (i.e., directory) where our images will be stored.

In [ ]:
import glob
import boto3
import os
import pandas as pd

bucket = "tmeservy-mldata" #replace this with your bucket name
prefix = "images"

We will download a sample set of images scraped from the internet. For your convenience these have been zipped up. We will use the wget shell command to get the data and then the unzip command to save those files in our local file system within SageMaker.

In [ ]:
!wget https://www.ishelp.info/data/images.zip -O images.zip
# Uncompressing
!unzip -qU -o images.zip

You may want to explore the image files that you now have on your local file system. To save them out to S3, we will get the S3 client from boto and then use the upload_file method as we have done in previous sections of the course.

In [ ]:
#upload the files to the S3 bucket
images = glob.glob("images/*.jpg")
for filename in images:
    boto3.Session().resource('s3').Bucket(bucket).upload_file(filename,f'{prefix}/{os.path.basename(filename)}')

### Using Rekognition to Detect Faces and Emotions - Single Image

Rekognition has a lot of different methods that can be used to analyze images and videos. These methods use pre-trained models to identify objects in the images. We first need to get access to a Rekognition client and then we simply pass an S3 object URI to the detect_faces method.

The detect faces method will find any faces for individuals in the image and will return their age range, gender, whether they are smiling, have a beard, etc. It will also associated artifacts like whether or not the individual has a beard, is wearing sunglasses, eyeglasses, etc. One of the useful states that it returns is the emotion of the individual.

In [ ]:
client = boto3.client('rekognition')

In [ ]:

response = client.detect_faces(
    Image={
        'S3Object': {
            'Bucket': bucket,
            'Name': f'{prefix}/3_1238858492653551617.jpg'
        }
    },
    Attributes=[
        'ALL',
    ]
)

In [ ]:
#print out the results
response

We can then extract the information that we are interested in from this JSON object using the approaches that we've learned previously.

### Using Rekognition to Detect Faces and Emotions - Multiple Images

If we want to process multiple files, we can simply get a list of the files that we want to process from S3. We'll first get the s3 client from boto and then get the files from the bucket and "directory" that we want by specifying the prefix that we want to use.

In [ ]:

s3 = boto3.resource('s3')
my_bucket = s3.Bucket(bucket)
files = my_bucket.objects.filter(Prefix='images')
    

The following code iterates through all of the files that are returned. Notice we filter out any directories.

In [ ]:
for file in files:
    #ignore directories
    if file.key[-1] == "/":
        continue
    print(file)

As you can see, this next code block iterates through all of the files in the list and makes a call for each one to the detect_faces method. It then takes the results and converts them into a pandas dataframe.

In [ ]:
df = pd.DataFrame([])
i=0

for file in files:
    #ignore directories
    if file.key[-1] == "/":
        continue

    #call rekognition for this next file
    response = client.detect_faces(
        Image={
            'S3Object': {
                'Bucket': file.bucket_name,
                'Name': file.key
            }
        },
        Attributes=[
            'ALL',
        ]
    )
    
    #now add all of the facial features for every person found in the photo
    for fd in response["FaceDetails"]:
        i+=1        
        df.loc[i,'File'] = file.key
        df.loc[i,'PersonID'] = i
        df.loc[i,'AgeRange-Low'] = fd["AgeRange"]["Low"]
        df.loc[i,'AgeRange-High'] = fd["AgeRange"]["High"]
        df.loc[i,'Smile'] = fd["Smile"]["Value"]
        df.loc[i,'Gender'] = fd["Gender"]["Value"]
        df.loc[i,'Emotion'] = fd["Emotions"][0]["Type"] #get dominant emotion
        df.loc[i,'Emotion-Confidence'] = fd["Emotions"][0]["Confidence"] #get dominant emotion

Now we can display the results. Notice how we made sure to add the name of the file to the dataframe so that we can eventually associate these results with the correct rows in out machine learning models.

In [ ]:
df

### Summarizing results from Rekognition Using Aggregate Functions

So, you've now processed all of the files with Rekognition and we have results. However, we often will need to consolidate this data into a single row per record in our machine learning model. To do that, we can use some built in python and pandas functions to summarize the data for each file. 

One way to aggregate data is to use functions such as count, min, make, etc. The following code first creates a data frame and then adds summary information by grouping on the filename for the file that was processed. 

In [ ]:
df2 = pd.DataFrame([])

#summarize stats per file
#aggregate functions include min, max, mean, count, and more.
df2['Count-People'] = df.groupby('File')['PersonID'].count()
df2['Avg-AgeRange-Low'] = df.groupby('File')['AgeRange-Low'].min()
df2['Avg-AgeRange-High'] = df.groupby('File')['AgeRange-High'].max()
df2['Count-Smile'] = df[df['Smile']==True].groupby('File')['Smile'].count()
df2['Percent-Smile'] = df2['Count-Smile']/df2['Count-People']

#print out 
df2


Pandas also has the ability to specify one or more aggregate functions all at once but using the agg function. The following code illustrates how we could calculate many different summary statistics all at once.

In [ ]:
df.groupby('File').agg({'PersonID': ['count'],
                       'AgeRange-Low': ['count','mean', 'min', 'max'],
                       'AgeRange-High': ['count','mean', 'min', 'max'],
                       'Smile': ['count'],
                       })

The downside to this approach is that we don't have names associated with each of the new columns. We can use a slightly different approach to name the columns as we generate the aggregate values. Notice that you can specify the a name for the new column (uses the name of the variable on the left side of the assignment operator) if you use the NamedAgg method.

In [ ]:
df3 = df.groupby('File').agg(
    NumPeople=pd.NamedAgg(column='PersonID', aggfunc='count'),
    AgeRange_Low=pd.NamedAgg(column='AgeRange-Low', aggfunc='min'),
    AgeRange_High=pd.NamedAgg(column='AgeRange-High', aggfunc='max'),
                      )
df3

That works pretty well for summarizing numeric variables/columns but how do you handle categorical variables like the emotion displayed on the different faces? Luckily, we can use the value_counts() method to create separate columns and the associated counts of each of the different labels that might appear in the emotions column. We can also specify a default value if a specific file doesn't have a particular emotion.

In [ ]:
df.groupby('File')['Emotion'].value_counts().unstack().fillna(0)


Now we need to take all the different summarized data and combine it into one data frame. We'll use the merge function to take and existing data frame and combine it with new columns of information. Note how we use the File field to make sure that we combine the correct data together.

In [ ]:
#start with df3 and add the count of different emotions and genders that were found in each of the files
final = df3.merge(df.groupby('File')['Emotion'].value_counts().unstack().fillna(0), on='File')
final = final.merge(df.groupby('File')['Gender'].value_counts().unstack().fillna(0),on='File')
final

Once we've generated this information, we can then save it to a csv for use in our models.

In [ ]:
final.to_csv("faces.csv")

### Using Rekognition to Detect and Label Objects

Using a similar approach, we can detect other objects within images. Rekognition refers to these other objects as labels. Thus, we will use the detect_labels methods to process our files. Notice the parameter where we specify that we only want up to 10 labels returned. Rekognition returns the 10 labels with the highest confidence scores.

In [ ]:
df = pd.DataFrame([])
i=0

#files = list(files)[:2]

for file in files:
    #ignore directories
    if file.key[-1] == "/":
        continue

    #print the file
    #print(file)
      
    #call rekognition for this next file
    response = client.detect_labels(
        Image={
            'S3Object': {
                'Bucket': file.bucket_name,
                'Name': file.key
            }
        }, MaxLabels=10
    )
    
    #now add all of the labels found in this photo
    for lb in response["Labels"]:
        i+=1 
        df.loc[i,'File'] = file.key
        df.loc[i,'LabelID'] = i
        df.loc[i,'Name'] = lb["Name"]
        df.loc[i,'Confidence'] = lb["Confidence"]

In [ ]:
df

As you can see, the results are a little bit simpler as we chose to only extract the name of the label and the confidence score. When you use labels in machine learning models you will often just include the count of the label. Of course, we could filter out labels that were less than a certain confidence score if we wanted to. We will use the value_counts function and the unstack function to create data that we could use in our machine learning models.

In [ ]:
df_labels = df.groupby('File')['Name'].value_counts().unstack().fillna(0)
df_labels

Notice that we have 132 different columns and we don't get to see the names of all of the columns. However, we can simply list out the names of the columns using the list function.

In [ ]:
#look at the names of the columns 
list(df_labels.columns)

Sometimes we are only interested in capturing when an image contains a specific label. Assuming it was included in the results returned from Rekognition, we can simply apply filtering to our dataset before we group by the filename as seen in the following example:

In [ ]:
df[df['Name']=='Apparel'].groupby('File')['Name'].value_counts().unstack().fillna(0)

Of course, if this data were of interest to our model we would combine this information with other relevant information and save it out to be used with our machine learning models.

### Using Rekognition to Identify Text Found in Images

Rekognition can also identify text that this found in images. The following code example demonstrates how the detect_text method can be used to extract text from images. Since this process is a bit more computational intensive, we will only process a few of our images by converting our files collection into a list and then specifying to only use a few images.

In [ ]:
df = pd.DataFrame([])
i=0

for file in list(files)[:10]:
    #ignore directories
    if file.key[-1] == "/":
        continue

    #print the file
    #print(file)
      
    #call rekognition for this next file
    response = client.detect_text(
        Image={
            'S3Object': {
                'Bucket': file.bucket_name,
                'Name': file.key
            }
        }
    )
    
    #now add all of the labels found in this photo
    for td in response["TextDetections"]:
        i+=1 
        #print(td)
        df.loc[i,'File'] = file.key
        df.loc[i,'TextID'] = i
        df.loc[i,'Text'] = td["DetectedText"]
        df.loc[i,'Confidence'] = td["Confidence"]


In [ ]:
df